## Day 24

https://adventofcode.com/2024/day/24

### Part 1

In [1]:
def read_input_24(filename):
    f = open(filename)
    l = f.read().split("\n\n")
    values = {}
    for v in l[0].split("\n"):
        vv = v.split(": ")
        values[vv[0]] = int(vv[1])
    network = {}
    for v in l[1].strip().split("\n"):
        vv = v.split(" -> ")
        inp = vv[0].split()
        network[vv[1]] = ((inp[0],inp[2]),inp[1])
    return values, network

In [2]:
def evaluate(node, values, network):
    '''Recursive evaluation of a network node, until an actual value is found'''
    if node in values:
        return values[node]
    (in1, in2), op = network[node]
    if op=="AND":
        return evaluate(in1, values, network) & evaluate(in2, values, network)
    elif op=="OR":
        return evaluate(in1, values, network) | evaluate(in2, values, network)
    elif op=="XOR":
        return evaluate(in1, values, network) ^ evaluate(in2, values, network)

def part1(filename):
    values, network = read_input_24(filename)
    output = {}
    for node in network:
        if node.startswith("z"):
            output[node] = evaluate(node, values, network)
    return sum([ 2**n * output[z] for n,z in enumerate( sorted(output.keys()) ) ])

In [3]:
print("Test 1-1:",part1("examples/example24-1.txt"))
print("Test 1-2:",part1("examples/example24-2.txt"))
print("Part 1  :",part1("AOC2024inputs/input24.txt"))

Test 1-1: 4
Test 1-2: 2024
Part 1  : 46362252142374


### Part 2

the netwowork is supposed to work as a summer, so I suppose it will be a chain of maby single bit summers in chain, each returning the XOR of the input bit and the possible carry-over:

https://en.wikipedia.org/wiki/Adder_(electronics)

A single summing block should therefore have this structure:

https://en.wikipedia.org/wiki/Adder_(electronics)#/media/File:Full-adder_logic_diagram.svg

In [73]:
def summer(X, Y, C):
    a = X ^ Y
    b = X & Y
    S = C ^ a
    c = C & a
    T = b | c
    return S, T

### Solution

I will render the network, identify the bit summer blocks, and search for the output wiring inversion. I will use the bit difference between the true additiona and the network result to guide the (manual) search

In [74]:
import graphviz as gv

values, network = read_input_24("AOC2024inputs/input24.txt")

G = gv.Digraph()

for out,((in1,in2),op) in network.items():
    if op=="AND":
        color = "green"
    elif op== "OR":
        color = "blue"
    elif op== "XOR":
        color = "red"

    G.edge(in1, out, label=op, color=color)
    G.edge(in2, out, label=op, color=color)

G.render("visualisation/day24", format="pdf")

'visualisation/day24.pdf'

In [141]:
def setvalues(x,y,values):
    # reset values
    for node in values:
        if node.startswith("x"):
            values[node] = 0
        if node.startswith("y"):
            values[node] = 0
    for i,bx in enumerate(bin(x)[2:][::-1]):
        values["x"+str(i).zfill(2)] = int(bx)
    for i,by in enumerate(bin(y)[2:][::-1]):
        values["y"+str(i).zfill(2)] = int(by)
    return values

def add(values,network):
    output = {}
    for node in network:
        if node.startswith("z"):
            output[node] = evaluate(node, values, network)
    return sum([ 2**n * output[z] for n,z in enumerate( sorted(output.keys()) ) ])

In [229]:
def setswaps(network):
    swaps = [ ('z06', 'jmq'), ('z13', 'gmh'), ('z38', 'qrh'), ('cbd', 'rqf') ]
    nodes = []
    for s1,s2 in swaps:
        network[s1], network[s2] = network[s2], network[s1]
        nodes.append(s1)
        nodes.append(s2)
    print(",".join(sorted(nodes)),"\n")
    return network

def print_sum(Xv,Yv,Zv,Zn):
    print("X    "+bin(Xv)[2:])
    print("Y    "+bin(Yv)[2:])
    print(50*"-")
    print("Zv ",bin(Zv)[2:])
    print(50*"-")
    print("Zn ",bin(Zn)[2:])
    print(50*"-")
    print(f"Z^Z {bin(Zn^Zv)[2:]:60s}")

values, network = read_input_24("AOC2024inputs/input24.txt")
network = setswaps(network)

X = [ bit for node,bit in values.items() if node.startswith("x")]
Y = [ bit for node,bit in values.items() if node.startswith("y")]

Xv = sum([ 2**n * x for n,x in enumerate(X) ])
Yv = sum([ 2**n * x for n,x in enumerate(Y) ])

Zv = Xv + Yv
Zn = add(values,network)

print_sum(Xv,Yv,Zv,Zn)

cbd,gmh,jmq,qrh,rqf,z06,z13,z38 

X    101101000011011001101001111001001111011001101
Y    100110110001111000000010101010110100000011001
--------------------------------------------------
Zv  1010011110101010001101100100100000011011100110
--------------------------------------------------
Zn  1010011110101010001101100100100000011011100110
--------------------------------------------------
Z^Z 0                                                           


In [231]:
values, network = read_input_24("AOC2024inputs/input24.txt")
network = setswaps(network)

x = 2**45-1
y = 2**45-1

values = setvalues(x,y,values)
z1 = add(values,network)
z2 = x+y
print(z1,z2,z1-z2, "\n")
print_sum(x,y,z1,z2)

cbd,gmh,jmq,qrh,rqf,z06,z13,z38 

70368744177662 70368744177662 0 

X    111111111111111111111111111111111111111111111
Y    111111111111111111111111111111111111111111111
--------------------------------------------------
Zv  1111111111111111111111111111111111111111111110
--------------------------------------------------
Zn  1111111111111111111111111111111111111111111110
--------------------------------------------------
Z^Z 0                                                           


In [228]:
import random

values, network = read_input_24("AOC2024inputs/input24.txt")

network = setswaps(network)

for _ in range(100):
    x = random.randint(0,2**45-1)
    y = random.randint(0,2**45-1)
    values = setvalues(x,y,values)
    z1 = add(values,network)
    z2 = x+y
    print(z1,z2,z1-z2)

cbd,gmh,jmq,qrh,rqf,z06,z13,z38 

30513712423716 30513712423716 0
12523904181292 12523904181292 0
47026374501393 47026374501393 0
18618047976406 18618047976406 0
16499586715371 16499586715371 0
29036062661011 29036062661011 0
41292660084731 41292660084731 0
59125119288134 59125119288134 0
35415479262547 35415479262547 0
38009736459269 38009736459269 0
49939617912667 49939617912667 0
38873151708299 38873151708299 0
29866508057991 29866508057991 0
30010915742513 30010915742513 0
13525013331860 13525013331860 0
40830187479518 40830187479518 0
31410838938951 31410838938951 0
63139350969639 63139350969639 0
13807633179199 13807633179199 0
30337239366887 30337239366887 0
37229438387170 37229438387170 0
57645533379762 57645533379762 0
40606006925271 40606006925271 0
11737145198619 11737145198619 0
25259464422432 25259464422432 0
24616052903430 24616052903430 0
44880366862096 44880366862096 0
46397643537352 46397643537352 0
32015314264130 32015314264130 0
38531099224711 38531099224711 0
252704